In [1]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader, Dataset
from torch.optim import AdamW
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
import pandas as pd

C:\Users\DKpro\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
df = pd.read_csv("Fake/Fake.csv")

In [3]:
df

,title,text,subject,date
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017"
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017"
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017"
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017"
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017"
...,...,...,...,...
23476,McPain: John McCain Furious That Iran Treated ...,21st Century Wire says As 21WIRE reported earl...,Middle-east,"January 16, 2016"
23477,JUSTICE? Yahoo Settles E-mail Privacy Class-ac...,21st Century Wire says It s a familiar theme. ...,Middle-east,"January 16, 2016"
23478,Sunnistan: US and Allied ‘Safe Zone’ Plan to T...,Patrick Henningsen 21st Century WireRemember ...,Middle-east,"January 15, 2016"
23479,How to Blow $700 Million: Al Jazeera America F...,21st Century Wire says Al Jazeera America will...,Middle-east,"January 14, 2016"


In [3]:
model = CLIPTextClassifier()
print(model)

CLIPTextClassifier(
  (clip): CLIPModel(
    (text_model): CLIPTextTransformer(
      (embeddings): CLIPTextEmbeddings(
        (token_embedding): Embedding(49408, 512)
        (position_embedding): Embedding(77, 512)
      )
      (encoder): CLIPEncoder(
        (layers): ModuleList(
          (0-11): 12 x CLIPEncoderLayer(
            (self_attn): CLIPSdpaAttention(
              (k_proj): Linear(in_features=512, out_features=512, bias=True)
              (v_proj): Linear(in_features=512, out_features=512, bias=True)
              (q_proj): Linear(in_features=512, out_features=512, bias=True)
              (out_proj): Linear(in_features=512, out_features=512, bias=True)
            )
            (layer_norm1): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
            (mlp): CLIPMLP(
              (activation_fn): QuickGELUActivation()
              (fc1): Linear(in_features=512, out_features=2048, bias=True)
              (fc2): Linear(in_features=2048, out_features=512, bias

In [4]:
from transformers import CLIPModel, CLIPTokenizer
import torch.nn as nn
import torch

class CLIPTextClassifier(nn.Module):
    def __init__(self, pretrained="openai/clip-vit-base-patch32"):
        super().__init__()
        self.clip = CLIPModel.from_pretrained(pretrained)

        for p in self.clip.vision_model.parameters():
            p.requires_grad = False
        for p in self.clip.visual_projection.parameters():
            p.requires_grad = False

        hidden = self.clip.config.projection_dim
        self.classifier = nn.Linear(hidden, 1)

        self.tokenizer = CLIPTokenizer.from_pretrained(pretrained)

    def forward(self, texts):
        enc = self.tokenizer(
            texts,
            padding=True,
            truncation=True,
            max_length=77,
            return_tensors="pt"
        ).to(self.classifier.weight.device)

        text_embeds = self.clip.get_text_features(
            input_ids=enc["input_ids"],
            attention_mask=enc["attention_mask"]
        )

        logits = self.classifier(text_embeds)
        return logits.squeeze(1)


In [7]:
import pandas as pd
from torch.utils.data import Dataset, DataLoader

def load_imdb_data(fake_csv: str, true_csv: str):
    fk = pd.read_csv(fake_csv)
    tr = pd.read_csv(true_csv)
    texts  = fk["text"].tolist() + tr["text"].tolist()
    labels = [0]*len(fk) + [1]*len(tr)
    return texts, labels

class NewsDataset(Dataset):
    def __init__(self, texts, labels):
        self.texts, self.labels = texts, labels
    def __len__(self):            
        return len(self.texts)
    def __getitem__(self, idx):   
        return self.texts[idx], self.labels[idx]

In [8]:
import torch.optim as optim
from sklearn.model_selection import train_test_split
from tqdm import tqdm

texts, labels = load_imdb_data("Fake/Fake.csv", "True/True.csv")
tr_x, val_x, tr_y, val_y = train_test_split(texts, labels, test_size=0.2, random_state=42, stratify=labels)

BATCH = 16
train_dl = DataLoader(NewsDataset(tr_x, tr_y), batch_size=BATCH, shuffle=True)
val_dl   = DataLoader(NewsDataset(val_x, val_y), batch_size=BATCH)

device = "cuda" if torch.cuda.is_available() else "cpu"
model  = CLIPTextClassifier().to(device)

criterion = nn.BCEWithLogitsLoss()
optimizer = optim.AdamW(filter(lambda p: p.requires_grad, model.parameters()),lr=2e-5)

EPOCHS = 3
for epoch in range(EPOCHS):
    model.train()
    running_loss = 0
    for batch_texts, batch_labels in tqdm(train_dl, desc=f"Epoch {epoch+1}/{EPOCHS}"):
        batch_labels = batch_labels.float().to(device)
        logits = model(batch_texts)
        loss   = criterion(logits, batch_labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * len(batch_labels)

    avg_loss = running_loss / len(train_dl.dataset)

    model.eval()
    correct = total = 0
    with torch.no_grad():
        for batch_texts, batch_labels in val_dl:
            batch_labels = batch_labels.float().to(device)
            logits = model(batch_texts)
            preds  = (torch.sigmoid(logits) > 0.5).long()
            correct += (preds.cpu() == batch_labels.cpu()).sum().item()
            total   += len(batch_labels)
    acc = correct / total
    print(f"[Epoch {epoch+1}] train_loss={avg_loss:.4f}  val_acc={acc:.3%}")


Epoch 1/3: 100%|███████████████████████████████████████████████████████████████████| 2245/2245 [10:05<00:00,  3.71it/s]


[Epoch 1] train_loss=0.0082  val_acc=99.900%


Epoch 2/3: 100%|███████████████████████████████████████████████████████████████████| 2245/2245 [09:40<00:00,  3.87it/s]


[Epoch 2] train_loss=0.0053  val_acc=99.944%


Epoch 3/3: 100%|███████████████████████████████████████████████████████████████████| 2245/2245 [09:39<00:00,  3.87it/s]


[Epoch 3] train_loss=0.0019  val_acc=100.000%


In [9]:
import os, torch

SAVE_DIR = "./clip_text_fake_news"
os.makedirs(SAVE_DIR, exist_ok=True)

torch.save(
    {
        "model_state": model.state_dict(),
        "optimizer_state": optimizer.state_dict(),
        "epoch": epoch + 1,
    },
    os.path.join(SAVE_DIR, "pytorch_model.bin")
)

model.tokenizer.save_pretrained(SAVE_DIR)

('./clip_text_fake_news\\tokenizer_config.json',
 './clip_text_fake_news\\special_tokens_map.json',
 './clip_text_fake_news\\vocab.json',
 './clip_text_fake_news\\merges.txt',
 './clip_text_fake_news\\added_tokens.json')

In [10]:
import torch
from transformers import CLIPTokenizer

def load_local_model(ckpt_dir, device="cpu"):
    model = CLIPTextClassifier()
    sd = torch.load(os.path.join(ckpt_dir, "pytorch_model.bin"), map_location=device)
    model.load_state_dict(sd["model_state"])
    model.to(device).eval()

    tokenizer = CLIPTokenizer.from_pretrained(ckpt_dir)
    return model, tokenizer

model, tokenizer = load_local_model("./clip_text_fake_news", device="cpu")

C:\Users\DKpro\AppData\Local\Temp\ipykernel_21088\2959785651.py:6: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  sd = torch.load(os.path.join(ckpt_dir, "pytorch_model.bin"),

In [12]:
model.eval()

def predict(text, model=model, device=device):
    with torch.no_grad():
        logits = model([text])
        prob   = torch.sigmoid(logits)[0].item()

    label = "true" if prob >= 0.5 else "fake"
    return label, prob

sample = (
"""From the moment a Norfolk Southern ‘bomb train’ derailed in East Palestine, OH, on February 3, 2023, traumatized and chemically exposed residents became another political football to be kicked around by Republicans, Democrats, and the media. Nearly two years since the avoidable catastrophe that changed their lives forever, residents in and around East Palestine and their families have been left to live in a toxic “sacrifice zone.” Like in 2020, the majority of voters in this part of Ohio and Pennsylvania will likely vote for Donald Trump in 2024, though plenty have given up on the whole system. In this on-the-ground documentary report, TRNN Editor-in-Chief Maximillian Alvarez and Steve Mellon from the Pittsburgh Union Progress go to East Palestine to speak with residents face to face, deep in the heart of so-called “Trump Country,” and what they find is a stark reminder that working-class communities have way more in common than corporate media and corporate politicians want us to believe."""
)

label, conf = predict(sample)
print(f"{label}  (confidence={conf:.3f})")


true  (confidence=0.999)


# Vision

In [2]:
import os, glob, random
import torch, torch.nn as nn, torch.optim as optim
from torch.utils.data import Dataset, DataLoader, Subset
from transformers import CLIPModel, CLIPImageProcessor
from PIL import Image
from tqdm import tqdm

def set_seed(seed=42):
    random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)

class FrameDataset(Dataset):
    IMG_EXT = {".jpg", ".jpeg", ".png", ".bmp", ".gif", ".webp"}

    def __init__(self, root: str):
        self.paths, self.labels = [], []
        for lbl, sub in enumerate(["Real", "Fake"]):
            folder = os.path.join(root, sub)
            for p in glob.glob(os.path.join(folder, "*")):
                if os.path.splitext(p)[1].lower() in self.IMG_EXT:
                    self.paths.append(p)
                    self.labels.append(lbl)

    def __len__(self): return len(self.paths)

    def __getitem__(self, idx):
        img = Image.open(self.paths[idx]).convert("RGB")
        return img, self.labels[idx]


def collate_pil(batch):
    imgs, labels = zip(*batch)
    return list(imgs), torch.tensor(labels)

class CLIPVisionClassifier(nn.Module):
    def __init__(self, ckpt="openai/clip-vit-base-patch32"):
        super().__init__()
        self.clip = CLIPModel.from_pretrained(ckpt)

        # freeze everything except vision encoder → classifier
        self.clip.text_model.requires_grad_(False)
        self.clip.text_projection.requires_grad = False
        self.clip.logit_scale.requires_grad = False

        hid = self.clip.config.projection_dim          # 512
        self.classifier = nn.Linear(hid, 1)            # binary logit
        self.processor = CLIPImageProcessor.from_pretrained(ckpt)

    def _encode(self, pil_list):
        batch = self.processor(images=pil_list,
                               return_tensors="pt").to(self.classifier.weight.device)
        return self.clip.get_image_features(pixel_values=batch.pixel_values)

    def forward(self, x):
        feats = x if isinstance(x, torch.Tensor) else self._encode(x)
        return self.classifier(feats).squeeze(1)
        
@torch.no_grad()
def evaluate(model, loader, criterion, device):
    model.eval()
    loss_sum = correct = n = 0
    for imgs, y in tqdm(loader, leave=False):
        y = y.float().to(device)
        logits = model(imgs)
        loss_sum += criterion(logits, y).item() * y.size(0)

        preds = (torch.sigmoid(logits) > 0.5).long()
        correct += (preds == y.long()).sum().item()
        n += y.size(0)
    return loss_sum / n, correct / n


def train_one_epoch(model, loader, criterion, optimiser, device):
    model.train()
    loss_sum = correct = n = 0
    for imgs, y in tqdm(loader, leave=False):
        y = y.float().to(device)
        logits = model(imgs)
        loss = criterion(logits, y)

        optimiser.zero_grad()
        loss.backward()
        optimiser.step()

        loss_sum += loss.item() * y.size(0)
        preds = (torch.sigmoid(logits) > 0.5).long()
        correct += (preds == y.long()).sum().item()
        n += y.size(0)
    return loss_sum / n, correct / n
    
if __name__ == "__main__":
    # config
    set_seed(42)
    root_dir   = "Final Dataset"
    batch_size = 32
    epochs     = 3
    lr         = 1e-5
    device     = "cuda" if torch.cuda.is_available() else "cpu"

    full_ds = FrameDataset(root_dir)
    idx = list(range(len(full_ds))); random.shuffle(idx)
    split = int(0.8 * len(idx))
    train_ds, val_ds = Subset(full_ds, idx[:split]), Subset(full_ds, idx[split:])

    train_dl = DataLoader(train_ds, batch_size=batch_size, shuffle=True,
                          num_workers=0, collate_fn=collate_pil, pin_memory=True)
    val_dl   = DataLoader(val_ds,   batch_size=batch_size, shuffle=False,
                          num_workers=0, collate_fn=collate_pil, pin_memory=True)

    model = CLIPVisionClassifier().to(device)
    criterion = nn.BCEWithLogitsLoss()
    optimiser = optim.AdamW(
        filter(lambda p: p.requires_grad, model.parameters()),
        lr=lr, weight_decay=1e-4
    )

    for ep in range(1, epochs + 1):
        print(f"\nEpoch {ep}/{epochs}")
        tr_loss, tr_acc = train_one_epoch(model, train_dl, criterion, optimiser, device)
        vl_loss, vl_acc = evaluate(model, val_dl, criterion, device)
        print(f"  train  loss={tr_loss:.4f}  acc={tr_acc:.2%}")
        print(f"  val    loss={vl_loss:.4f}  acc={vl_acc:.2%}")


Epoch 1/3


  train  loss=0.2124  acc=91.16%
  val    loss=0.0740  acc=97.63%

Epoch 2/3


  train  loss=0.0567  acc=97.81%
  val    loss=0.0995  acc=96.66%

Epoch 3/3


  train  loss=0.0371  acc=98.71%
  val    loss=0.0661  acc=97.63%


In [3]:
torch.save(model.state_dict(), "vision_deepfake.bin")

AttributeError: 'Image' object has no attribute 'to'

In [4]:
from PIL import Image
import torch

device = "cuda" if torch.cuda.is_available() else "cpu"

model = CLIPVisionClassifier().to(device)
model.load_state_dict(torch.load("vision_deepfake.bin", map_location=device))
model.eval()

test_img = Image.open("Final Dataset/Fake/0EJSVRSK2A.jpg").convert("RGB")

with torch.no_grad():
    logit = model([test_img])
    prob  = torch.sigmoid(logit).item()

print("fake" if prob > 0.5 else "real", prob)

C:\Users\DKpro\AppData\Local\Temp\ipykernel_7664\3987242799.py:8: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load("vision_deepfake.bin", map_l

fake 0.9992254972457886
